In [2]:
import json
import os
import django
import sys
sys.path.append('..')
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'SMARTROLL.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'SMARTROLL.settings')
django.setup()
from Manage.models import College,Term,Branch,Stream,Semester,Division,Batch,Subject,TimeTable,GPSCoordinates,Classroom,Schedule,Lecture,Link,PermanentSubject,SubjectChoices
from Session.models import Attendance,Session
from StakeHolders.models import NotificationSubscriptions,SuperAdmin,Admin,Teacher,Student,Profile
from django.contrib.auth import get_user_model
import datetime
import pandas as pd
import string

In [1]:
file_path='/home/manav1011/Documents/SMARTROLL_SSIP_2023/SMARTROLL/notebooks/sheets/electives.xlsx'

In [3]:
sheet_names = pd.ExcelFile(file_path).sheet_names
ML = pd.read_excel(file_path, sheet_name='ML')
ML_COUNT = ML['enrollment'].count()
print(f"ML Student Count - {ML_COUNT}")
BDA = pd.read_excel(file_path, sheet_name='BDA')
BDA_COUNT = BDA['enrollment'].count()
print(f"BDA Student Count - {BDA_COUNT}")
IS = pd.read_excel(file_path, sheet_name='IS')
IS_COUNT = IS['enrollment'].count()
print(f"IS Student Count - {IS_COUNT}")
MCWC = pd.read_excel(file_path, sheet_name='MCWC')
MCWC_COUNT = MCWC['enrollment'].count()
print(f"MCWC Student Count - {MCWC_COUNT}")
CC = pd.read_excel(file_path, sheet_name='CC')
CC_COUNT = CC['enrollment'].count()
print(f"CC Student Count - {CC_COUNT}")
MAD = pd.read_excel(file_path, sheet_name='MAD')
MAD_COUNT = MAD['enrollment'].count()
print(f"MAD Student Count - {MAD_COUNT}")
DF = pd.read_excel(file_path, sheet_name='DF')
DF_COUNT = DF['enrollment'].count()
print(f"DF Student Count - {DF_COUNT}")


ML Student Count - 149
BDA Student Count - 29
IS Student Count - 178
MCWC Student Count - 24
CC Student Count - 154
MAD Student Count - 18
DF Student Count - 160


In [4]:
def update_student_subjects(df, subject_name,student_subjects):
    for _, row in df.iterrows():
        student_key = (row['enrollment'], row['name'])  # Create a tuple of (enrollment, name)
        if student_key in student_subjects:
            student_subjects[student_key].append(subject_name)
        else:
            student_subjects[student_key] = [subject_name]


In [5]:
student_subjects = {}
update_student_subjects(ML, 'ML',student_subjects)
update_student_subjects(BDA, 'BDA',student_subjects)
update_student_subjects(IS, 'IS',student_subjects)
update_student_subjects(MCWC, 'MCWC',student_subjects)
update_student_subjects(CC, 'CC',student_subjects)
update_student_subjects(MAD, 'MAD',student_subjects)
update_student_subjects(DF, 'DF',student_subjects)

In [6]:
min_div_size = 60
max_div_size = 90
def calculate_divisions(student_count):    
    min_divisions = (student_count + max_div_size - 1) // max_div_size
    max_divisions = student_count // min_div_size        
    for divisions in range(min_divisions, max_divisions + 1):
        div_size = student_count // divisions
        if min_div_size <= div_size <= max_div_size:
            return divisions        
    return min_divisions

In [7]:
total_hours = {
    "ML":5,
    "BDA":5,
    "IS":5,
    "MCWC":3,
    "CC":3,
    "MAD":4,
    "DF":4,
}
total_student_count = len(student_subjects.keys())
approx_division_count = calculate_divisions(total_student_count)
# the total teaching our for each subject will be in multiplication of the approx division count

# for subject in total_hours:
#     total_hours[subject] *= approx_division_count
# total_hours

In [8]:
subject_groups = {}
for student, subject_group in student_subjects.items():
    # Convert subject_group to a tuple if it isn't one already
    subject_group_tuple = tuple(subject_group)
    if subject_group_tuple not in subject_groups:
        subject_groups[subject_group_tuple] = [student]
    else:
        subject_groups[subject_group_tuple].append(student)

In [9]:
for subject_group in subject_groups:
    print(f"Group - {subject_group} | Student Count {len(subject_groups[subject_group])}")

Group - ('ML', 'IS', 'CC', 'DF') | Student Count 138
Group - ('ML', 'IS', 'CC', 'MAD') | Student Count 6
Group - ('ML', 'IS', 'MCWC', 'MAD') | Student Count 2
Group - ('ML', 'IS', 'MCWC', 'DF') | Student Count 3
Group - ('BDA', 'IS', 'MCWC', 'MAD') | Student Count 9
Group - ('BDA', 'IS', 'CC', 'DF') | Student Count 9
Group - ('BDA', 'IS', 'CC', 'MAD') | Student Count 1
Group - ('BDA', 'IS', 'MCWC', 'DF') | Student Count 10


In [10]:
divisions = {}
for i in range(approx_division_count):
    divisions[string.ascii_uppercase[i]] = []
divisions

{'A': [], 'B': []}

In [11]:
remaining_students_count = total_student_count
for division in divisions:        
    # check the count of students in current division
    students_count_in_div = len(divisions[division])
    capacity_remained_in_div = max_div_size-students_count_in_div
    if capacity_remained_in_div >= remaining_students_count:
        # insert all the remaining students in current div
        for subject_group,students in subject_groups.items():
            divisions[division] += students
    else:
        # iterate through the subject groups
        # check if any available subject group has the student count greater than the max_div_count
        subject_group_gt_max_div_count= list(filter(lambda subject_group: len(subject_groups[subject_group]) > max_div_size,subject_groups))
        if subject_group_gt_max_div_count:
            # move the first max_div_count students to this division
            divisions[division] = subject_groups[subject_group_gt_max_div_count[0]][:max_div_size]
            subject_groups[subject_group_gt_max_div_count[0]] = subject_groups[subject_group_gt_max_div_count[0]][max_div_size:]
            # also reduce the remaining student count            
            remaining_students_count -= max_div_size
            # now this div's full so move let's move to next div            
            continue
            

In [ ]:
# Now to sort the students according to their enrollment in each divisions
for division in divisions:
    divisions[division]=sorted(divisions[division], key=lambda student: student[0])
divisions

{'A': [(200280107145, 'Prajapati Jay Kumar Dashrathbhai '),
  (210280107001, 'Ratadiya Mayur Devsibhai '),
  (210280107003, 'Taral Bambhania'),
  (210280107004, 'Darshil Nareshbhai Patel'),
  (210280107005, 'Akshar Chauhan'),
  (210280107006, 'Patel Nisarg Surendrakumar'),
  (210280107007, 'DOBARIYA HAPPY RAMNIKBHAI '),
  (210280107008, 'Meet Soni '),
  (210280107009, 'Nirali Darji'),
  (210280107010, 'Devmurari Maulik Vijaybhai'),
  (210280107011, 'Justin George'),
  (210280107012, 'Patel Dhruvit Vasantbhai '),
  (210280107013, 'Halpati Divya DipakKumar '),
  (210280107014, 'Jiyani Hasti Mukeshbhai'),
  (210280107015, 'Astha Hareshbhai Viradiya '),
  (210280107016, 'Kaman Devanshi KaushikKumar '),
  (210280107017, 'Naik Chinmay Dharmesh'),
  (210280107018, 'Patel Chirag Sunilbhai'),
  (210280107019, 'PRAJAPATI HARSH DHIRENDRAKUMAR '),
  (210280107020, 'KERALIYA KARN NAROTTAMBHAI'),
  (210280107022, 'Henish Kantibhai Naliyadhara '),
  (210280107024, 'Jha Kunjbihari Vinodanand '),
  (21

In [15]:
# Sample data for student groups and their counts
student_groups = {
    ('ML', 'IS', 'CC', 'DF') :138,
    ('ML', 'IS', 'CC', 'MAD') :6,
    ('ML', 'IS', 'MCWC', 'MAD') :2,
    ('ML', 'IS', 'MCWC', 'DF') :3,
    ('BDA', 'IS', 'MCWC', 'MAD') :9,
    ('BDA', 'IS', 'CC', 'DF') :9,
    ('BDA', 'IS', 'CC', 'MAD') :1,
    ('BDA', 'IS', 'MCWC', 'DF') :10,
}

# GTU minimum teaching hours for each subject
total_hours = {
    "ML": 4,
    "BDA": 4,
    "IS": 4,
    "MCWC": 3,
    "CC": 3,
    "MAD": 3,
    "DF": 3,
}

# Initialize two divisions
division_1 = {}
division_2 = {}
# Initialize teaching hours tracker for each subject in both divisions
teaching_hours_div_1 = {subject: 0 for subject in total_hours}
teaching_hours_div_2 = {subject: 0 for subject in total_hours}

# Initialize student counters for both divisions
division_1_students = 0
division_2_students = 0

# Define the student limits per division
min_students_per_division = 60
max_students_per_division = 90

# Function to calculate the penalty score for deviating from the ideal teaching hours
def calculate_penalty(division_hours):
    penalty = 0
    for subject, ideal_hours in total_hours.items():
        effective_hours = max(division_hours[subject], ideal_hours)  # Apply minimum threshold
        penalty += abs(effective_hours - ideal_hours)
    return penalty

# Main loop to assign each group to a division
for subjects, count in student_groups.items():
    # For the group ('ML', 'IS', 'CC', 'DF') with 138 students:
    if count > max_students_per_division:
        # Split the group into two parts
        first_part = max_students_per_division - division_1_students
        second_part = count - first_part

        # Add the first part to Division 1
        division_1[subjects] = first_part
        division_1_students += first_part
        for subject in subjects:
            teaching_hours_div_1[subject] += first_part * total_hours[subject] / count
        
        # Add the second part to Division 2
        division_2[subjects] = second_part
        division_2_students += second_part
        for subject in subjects:
            teaching_hours_div_2[subject] += second_part * total_hours[subject] / count

    else:
        # Assign the entire group to Division 1 or Division 2 if it fits
        if division_1_students + count <= max_students_per_division:
            # Assign to Division 1
            division_1[subjects] = count
            division_1_students += count
            for subject in subjects:
                teaching_hours_div_1[subject] += count * total_hours[subject] / count
        else:
            # Assign to Division 2
            division_2[subjects] = count
            division_2_students += count
            for subject in subjects:
                teaching_hours_div_2[subject] += count * total_hours[subject] / count

# Output the results
print("Division 1 student groups and counts:", division_1)
print("Division 2 student groups and counts:", division_2)
print("\nTeaching hours in Division 1:", teaching_hours_div_1)
print("Teaching hours in Division 2:", teaching_hours_div_2)
print("Total Students in Division 1:", division_1_students)
print("Total Students in Division 2:", division_2_students)

Division 1 student groups and counts: {('ML', 'IS', 'CC', 'DF'): 90}
Division 2 student groups and counts: {('ML', 'IS', 'CC', 'DF'): 48, ('ML', 'IS', 'CC', 'MAD'): 6, ('ML', 'IS', 'MCWC', 'MAD'): 2, ('ML', 'IS', 'MCWC', 'DF'): 3, ('BDA', 'IS', 'MCWC', 'MAD'): 9, ('BDA', 'IS', 'CC', 'DF'): 9, ('BDA', 'IS', 'CC', 'MAD'): 1, ('BDA', 'IS', 'MCWC', 'DF'): 10}

Teaching hours in Division 1: {'ML': 2.608695652173913, 'BDA': 0, 'IS': 2.608695652173913, 'MCWC': 0, 'CC': 1.9565217391304348, 'MAD': 0, 'DF': 1.9565217391304348}
Teaching hours in Division 2: {'ML': 13.391304347826086, 'BDA': 16.0, 'IS': 29.391304347826086, 'MCWC': 12.0, 'CC': 10.043478260869566, 'MAD': 12.0, 'DF': 10.043478260869566}
Total Students in Division 1: 90
Total Students in Division 2: 88
